## CSC 580: Critical Thinking 2 - Predicting Future Sales
In a nutshell, *sales_data_test.csv* and *sales_data_test.csv* contain data that will be used to train a neural network to predict how much money can be expected form the future sale of new video games. The .csv files were retrieved from one of [Toni Esteves repos](https://github.com/toniesteves/adam-geitgey-building-deep-learning-keras/tree/master/03). 

The columns in the data are defined as follows:
- critic_rating : an average rating out of five stars
- is_action : tells us if this was an action game
- is_exclusive_to_us : tells us if we have an exclusiv deal to sell this game
- is_portable : tells us if this game runs on a handheld video game system
- is_role_playing : tells us if this is a role-playing game
- is_sequel : tells us if this game was a sequel to an earlier video game and part of an ongoing series
- is_sports : tells us if this was a sports game
- suitable_for_kids : tells us if this game is appropriate for all ages
- total_earning : tells us how much money the store has earned in total from selling the game to all customers
- unit_price : tells us for how much a single copy of the game retailed

In [17]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
import numpy as np
from keras.models import Sequential
from keras import layers 
from keras import activations
from keras import optimizers
from keras import losses

#### Step 1: Prepare the Dataset
The numerical data needs to be scaled for better network training

In [5]:
# Load the training and testing data
train_data = pd.read_csv("sales_data_training.csv")
test_data = pd.read_csv("sales_data_test.csv")

# Scale the data using sklearn
scaler = MinMaxScaler(feature_range=(0,1))
train_data_scaled = scaler.fit_transform(train_data)
test_data_scaled = scaler.fit_transform(test_data)

# Print out adjustment
print("Note: total_earnings values were scaled by multiplying by {:.10f} and adding {:.6f}".format(scaler.scale_[8], scaler.min_[8]))

# Create new DataFrames
df_train_scaled = pd.DataFrame(train_data_scaled, columns=train_data.columns.values)
df_test_scaled = pd.DataFrame(test_data_scaled, columns=test_data.columns.values)

# Save scaled data
df_train_scaled.to_csv("sales_data_training_scaled.csv", index=False)
df_test_scaled.to_csv("sales_data_testing_scaled.csv", index=False)


Note: total_earnings values were scaled by multiplying by 0.0000042367 and adding -0.153415


#### Part 2: Coding the Network

In [12]:
# Load the training data
training_data_df = pd.read_csv("sales_data_training_scaled.csv")

X = training_data_df.drop('total_earnings', axis=1).values
Y = training_data_df[['total_earnings']].values

In [23]:
# Model definition
model = Sequential(
    [
        layers.Input((9,)),
        layers.Dense(64, activation=activations.relu),
        layers.Dense(32, activation=activations.relu),
        layers.Dense(1, activation=activations.linear)
    ]
)

model.compile('adam', losses.binary_crossentropy)

#### Part 3: Training the Network

In [26]:
model.fit(X,Y,batch_size = 10, epochs = 50, verbose=2, shuffle=True)

Epoch 1/50
100/100 - 0s - loss: 0.5379
Epoch 2/50
100/100 - 0s - loss: 0.5378
Epoch 3/50
100/100 - 0s - loss: 0.5378
Epoch 4/50
100/100 - 0s - loss: 0.5377
Epoch 5/50
100/100 - 0s - loss: 0.5377
Epoch 6/50
100/100 - 0s - loss: 0.5377
Epoch 7/50
100/100 - 0s - loss: 0.5377
Epoch 8/50
100/100 - 0s - loss: 0.5376
Epoch 9/50
100/100 - 0s - loss: 0.5376
Epoch 10/50
100/100 - 0s - loss: 0.5376
Epoch 11/50
100/100 - 0s - loss: 0.5376
Epoch 12/50
100/100 - 0s - loss: 0.5376
Epoch 13/50
100/100 - 0s - loss: 0.5376
Epoch 14/50
100/100 - 0s - loss: 0.5376
Epoch 15/50
100/100 - 0s - loss: 0.5376
Epoch 16/50
100/100 - 0s - loss: 0.5376
Epoch 17/50
100/100 - 0s - loss: 0.5376
Epoch 18/50
100/100 - 0s - loss: 0.5376
Epoch 19/50
100/100 - 0s - loss: 0.5376
Epoch 20/50
100/100 - 0s - loss: 0.5376
Epoch 21/50
100/100 - 0s - loss: 0.5376
Epoch 22/50
100/100 - 0s - loss: 0.5376
Epoch 23/50
100/100 - 0s - loss: 0.5376
Epoch 24/50
100/100 - 0s - loss: 0.5376
Epoch 25/50
100/100 - 0s - loss: 0.5376
Epoch 26/